In [ ]:
import os
import json

carpeta_origen = r"C:\Users\migue\OneDrive\Escritorio\UAB_INTELIGENCIA_ARTIFICIAL\Tercer_Any\3B\Synthesis Project II\Limpieza\TW (Cruilla 2014-2024)"    
carpeta_destino = r"C:\Users\migue\OneDrive\Escritorio\UAB_INTELIGENCIA_ARTIFICIAL\Tercer_Any\3B\Synthesis Project II\Limpieza\CL - TW (Cruilla 2014-2024)"  

os.makedirs(carpeta_destino, exist_ok=True)

TWEET = [
    "id", "url", "fullText", "retweetCount","replyCount","likeCount",
    "quoteCount", "viewCount", "createdAt", "lang", "isReply", "conversationId",
    "isRetweet"
]
AUTHOR = [
    "userName", "followers", "following", "location"
]

for archivo in os.listdir(carpeta_origen):
    if archivo.endswith(".json"):
        ruta_archivo = os.path.join(carpeta_origen, archivo)

        with open(ruta_archivo, "r", encoding="utf-8") as f:
            datos = json.load(f)

        tweets_limpios = []
        ids_vistos = set()

        for tweet in datos:
            tweet_id = tweet.get("id")
            if tweet_id in ids_vistos:
                continue  
            ids_vistos.add(tweet_id)

            limpio = {campo: tweet.get(campo) for campo in TWEET}
            if "author" in tweet:
                limpio["author"] = {campo: tweet["author"].get(campo) for campo in AUTHOR}
            tweets_limpios.append(limpio)

        ruta_guardado = os.path.join(carpeta_destino, archivo)
        with open(ruta_guardado, "w", encoding="utf-8") as f:
            json.dump(tweets_limpios, f, ensure_ascii=False, indent=2)

        print(f"{archivo}: {len(datos)} → {len(tweets_limpios)} no duplicates")


TW2025-04-28_19.54h (Cruilla 2024).json: 1245 → 486 no duplicates
TW2025-04-28_20.12h (Cruilla 2024).json: 1245 → 486 no duplicates
TW2025-04-28_20.18h (Cruilla 2024).json: 1245 → 486 no duplicates
TW2025-04-28_20.20h (Cruilla 2024).json: 1063 → 415 no duplicates
TW2025-04-28_20.28h (Cruilla 2024).json: 441 → 185 no duplicates
TW2025-04-28_20.45h (Cruilla 2024).json: 262 → 108 no duplicates
TW2025-04-28_22.17h (Cruilla 2023).json: 710 → 316 no duplicates
TW2025-04-28_22.18h (Cruilla 2023).json: 538 → 288 no duplicates
TW2025-04-28_22.19h (Cruilla 2023).json: 651 → 288 no duplicates
TW2025-04-28_22.20h (Cruilla 2023).json: 491 → 227 no duplicates
TW2025-04-28_22.21h (Cruilla 2022).json: 506 → 217 no duplicates
TW2025-04-28_22.21h (Cruilla 2023).json: 491 → 227 no duplicates
TW2025-04-28_22.22h (Cruilla 2022).json: 506 → 217 no duplicates
TW2025-04-28_22.23h (Cruilla 2022).json: 410 → 175 no duplicates
TW2025-04-28_22.24h (Cruilla 2022).json: 410 → 175 no duplicates
TW2025-04-28_22.25h (

In [ ]:
import os
import json
import csv

carpeta_json_limpios = r"C:\Users\migue\OneDrive\Escritorio\UAB_INTELIGENCIA_ARTIFICIAL\Tercer_Any\3B\Synthesis Project II\Limpieza\CL - TW (Cruilla 2014-2024)"
csv_tweets = r"C:\Users\migue\OneDrive\Escritorio\UAB_INTELIGENCIA_ARTIFICIAL\Tercer_Any\3B\Synthesis Project II\Limpieza\all_tweets.csv"

filas = []

for archivo in os.listdir(carpeta_json_limpios):
    if archivo.endswith(".json"):
        ruta_archivo = os.path.join(carpeta_json_limpios, archivo)
        with open(ruta_archivo, "r", encoding="utf-8") as f:
            datos = json.load(f)

        for tweet in datos:
            fila = {campo: tweet.get(campo, "") for campo in TWEET}
            autor = tweet.get("author", {})
            for campo_autor in AUTHOR:
                fila[f"author_{campo_autor}"] = autor.get(campo_autor, "")
            filas.append(fila)

campos_csv = TWEET + [f"author_{campo}" for campo in AUTHOR]

with open(csv_tweets, "w", encoding="utf-8", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=campos_csv)
    writer.writeheader()
    writer.writerows(filas)

print(f"CSV creado con {len(filas)}")


CSV creado con 101569


In [50]:
import pandas as pd
df = pd.read_csv(csv_tweets, encoding="utf-8")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101569 entries, 0 to 101568
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                101569 non-null  int64  
 1   url               101569 non-null  object 
 2   fullText          101569 non-null  object 
 3   retweetCount      101569 non-null  int64  
 4   replyCount        101569 non-null  int64  
 5   likeCount         101569 non-null  int64  
 6   quoteCount        101569 non-null  int64  
 7   viewCount         15958 non-null   float64
 8   createdAt         101569 non-null  object 
 9   lang              101569 non-null  object 
 10  isReply           101569 non-null  bool   
 11  conversationId    101569 non-null  int64  
 12  isRetweet         101569 non-null  bool   
 13  author_userName   101569 non-null  object 
 14  author_followers  101569 non-null  int64  
 15  author_following  101569 non-null  int64  
 16  author_location   87

In [ ]:

def normalizar_fulltext(texto):
    if pd.isna(texto):
        return texto
    return texto.lower()

df["fullText"] = df["fullText"].apply(normalizar_fulltext)

In [ ]:
import pandas as pd
import re
import emoji


# ========= 1. ELIMINAR TWEETS DE REVENTA =========

patrones_reventa = [
    # Español
    r"\b(vendo|vendemos|vendiendo|se venden|vendo\s+\d+|vendo\s+una|vendo\s+varias)\s+(entradas?|abonos?)\b",
    r"\b(compro|compramos|comprando|busco|necesito)\s+(entradas?|abonos?)\b",
    r"\b(cambio|cambiamos|cambiando)\s+(entradas?|abonos?)\b",
    r"\b(tengo|ofrezco|dispongo de)\s+(entradas?|abonos?)\b",
    r"\b(sorteo|sorteamos|sorteando)\s+(entradas?|abonos?)\b",
    r"\b(gana|ganar|consigue|consigues)\s+(entradas?|abonos?)\b",
    r"\b(venc|vendo|busco|vendemos|venden)\b",

    # Catalán
    r"\b(venem|venc|venent|es venen)\s+(entrades?|abonaments?)\b",
    r"\b(compro|comprem|busco|necessito|necessitem)\s+(entrades?|abonaments?)\b",
    r"\b(canvio|canviem|canviant)\s+(entrades?|abonaments?)\b",
    r"\b(tinc|ofereixo|disposo de)\s+(entrades?|abonaments?)\b",
    r"\b(sorteig|sorteigem|sortegem|sortegen)\s+(entrades?|abonaments?)\b",
    r"\b(guanya|guanyar)\s+(entrades?|abonaments?)\b",

    # Inglés
    r"\b(sell|selling|resell|reselling|buy|buying)\s+(tickets?|passes|entries?)\b",
    r"\btickets?\s+(for sale|available|to sell|giveaway|give away)\b",
    r"\b(win|winning|get|claim)\s+(tickets?|passes|entries?)\b",
]

patron_reventa = '|'.join(patrones_reventa)
filtro_reventa = df["fullText"].str.contains(patron_reventa, case=False, na=False, regex=True)
df = df[~filtro_reventa]

# ========= 2. QUITAR DUPLICADOS POR ID =========

df = df.drop_duplicates(subset=["id"], keep="first")

# ========= 3. LIMPIAR TEXTOS: EMOJIS Y HASHTAGS =========

def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    texto = re.sub(r'http\S+|www.\S+', '', texto)
    texto = re.sub(r'https\S+|www.\S+', '', texto)
    # Quitar menciones @
    texto = re.sub(r'@\w+', '', texto)
    # Quitar hashtags
    texto = re.sub(r'#\w+', '', texto)
    texto = emoji.replace_emoji(texto, replace='')
    # Quitar espacios extra
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

df["fullText"] = df["fullText"].apply(limpiar_texto)

# ========= GUARDAR NUEVO CSV LIMPIO =========

df.to_csv(csv_tweets, index=False, encoding="utf-8")
print("CSV limpio guardado con éxito.")


C:\Users\migue\AppData\Local\Temp\ipykernel_29552\1771910675.py:33: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro_reventa = df["fullText"].str.contains(patron_reventa, case=False, na=False, regex=True)


CSV limpio guardado con éxito.


In [53]:
import pandas as pd
import re
csv_tweets = r"C:\Users\migue\OneDrive\Escritorio\UAB_INTELIGENCIA_ARTIFICIAL\Tercer_Any\3B\Synthesis Project II\Limpieza\all_tweets.csv"
df = pd.read_csv(csv_tweets, encoding="utf-8")

In [ ]:
marcas = [
    "cruillabcn", "barcelonaglobal", "btvnoticies", "lavanguardia", "diaridebcn", 
    "cacaolat", "estrelladamm", "rodalies", "catalannews", "movistar_es", 
    "elpaiscatalunya", "radioudeg", "324cat", "naciodigital", "spotify", 
    "on_economia", "on_economia_cat", "diariregio7", "elpeoriodico", "msn_es", "Cati_Tarragona", "beteve", 
    "MBecerraTour", "nuvol_com", "Canal21Ebre", "MPC_management", "elnacionalcat_e",
    "EPSI_UAB", "mundodeportivo", "cati_tarragona", "elnacionalcat", "los40",
    "CatalunyaPlural", "ClubRACC", "rac105", "AlDiaCat", "bnmallorca",
    "subnoise", "canal21ebre", "ebredigitalcat", "aguaitacat", "ElReferente",
    "actualidad_na", "ElNacionalWeb", "EuropaFMCat", "MWCapital", "laganzuaweb",
    "primerafilacat", 

]

df["author_userName"] = df["author_userName"].astype(str).str.lower()

es_marca = df["author_userName"].isin(marcas)

keywords_cuentas = ["turismo","turisme","info","news","diari", "diario ", "noticias", "info", "informacion", "new",
                    "radio", "barcelona", "tele", "es", "noticies", "tv", "magazine", "revista", "bcn",
                    "editorial", "hotel", "biblioteca", "spain", "racc", "web"]
es_cuenta_info = df["author_userName"].str.contains('|'.join(keywords_cuentas), na=False)

df_marcas = df[es_marca | es_cuenta_info][["url", "author_userName", 
                                           "replyCount"]].copy()

df = df[~(es_marca | es_cuenta_info)]

top_autores = df["author_userName"].value_counts().head(20)  # top 20

print("Autores con más tweets:")
print(top_autores)

print(f"Tweets eliminados: {len(df_marcas)}")
print(f"Tweets: {len(df)}")
print(df["author_userName"].unique())
print(df["author_following"].unique())

df["createdAt"] = pd.to_datetime(df["createdAt"], errors="coerce")
df["createdAt"] = df["createdAt"].dt.strftime("%d/%m/%Y %H:%M")

df.to_csv(r"C:\Users\migue\OneDrive\Escritorio\UAB_INTELIGENCIA_ARTIFICIAL\Tercer_Any\3B\Synthesis Project II\Limpieza\sin_compañias.csv", index=False, encoding="utf-8")


Autores con más tweets:
author_userName
yulianavk          203
amnistiacat         68
lluislazaro         61
canalte             55
lenadorfilms        55
elenanorabioso      43
mmarsuarez          42
nuriaar1969         41
farandularty        37
lufebrero           37
vinnicia7365        35
isidorajamett       34
albertmaruny        33
elisenda39          33
bunburyuniverso     31
oiacat              31
revers__en          30
jorge_morotxo       29
cronicaglobal       28
biancadevilar       28
Name: count, dtype: int64
Tweets eliminados: 13061
Tweets: 11382
['arasogranollers' 'archivocovid' 'daniel_spawn' ... 'helenagm' 'msolbun'
 'glauco42487234']
[1792  293  962 ... 2192 1275 2645]


C:\Users\migue\AppData\Local\Temp\ipykernel_29552\4155906267.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["createdAt"] = pd.to_datetime(df["createdAt"], errors="coerce")


In [55]:
df_marcas_con_respuestas = df_marcas[df_marcas["replyCount"] > 3].copy()

print(f"Tweets de marcas con respuestas: {len(df_marcas_con_respuestas)}")
print(df_marcas_con_respuestas[["url", "replyCount", "author_userName"]])
df_marcas_con_respuestas = df_marcas_con_respuestas[["url"]]
df_marcas_con_respuestas.to_csv(r"C:\Users\migue\OneDrive\Escritorio\UAB_INTELIGENCIA_ARTIFICIAL\Tercer_Any\3B\Synthesis Project II\Limpieza\compañias_respuestas.csv", index=False, encoding="utf-8")

Tweets de marcas con respuestas: 220
                                                     url  replyCount  \
30     https://x.com/cruillabcn/status/18161275900984...           5   
76     https://x.com/rodalies/status/1797531252460753276          17   
109    https://x.com/cruillabcn/status/18131334874796...           5   
118    https://x.com/discosdesalsa/status/18117493745...           9   
121    https://x.com/iAvrilSpain_/status/181167939351...           5   
...                                                  ...         ...   
21552  https://x.com/ClipsterBCN/status/7498774364987...           4   
22208  https://x.com/EstrellaDammCat/status/620560910...           4   
23252  https://x.com/EstrellaDammCat/status/618668342...          32   
23272  https://x.com/EstrellaDammCat/status/618384083...          26   
23285  https://x.com/EstrellaDammCat/status/618131553...          68   

       author_userName  
30          cruillabcn  
76            rodalies  
109         cruillabcn 

In [56]:
df_con_respuestas = df[df["replyCount"] > 3].copy()

print(f"Tweets de marcas con respuestas: {len(df_con_respuestas)}")
print(df_con_respuestas[["url", "replyCount", "author_userName"]])
df_con_respuestas = df_con_respuestas[["url"]]
df_con_respuestas.to_csv(r"C:\Users\migue\OneDrive\Escritorio\UAB_INTELIGENCIA_ARTIFICIAL\Tercer_Any\3B\Synthesis Project II\Limpieza\no_compañias_respuestas.csv", index=False, encoding="utf-8")

Tweets de marcas con respuestas: 82
                                                     url  replyCount  \
63      https://x.com/el_pais/status/1814617509405831661           8   
83     https://x.com/PlanetaAmaral/status/18124401785...           5   
156    https://x.com/MBComunidad/status/1811177199585...           5   
160       https://x.com/YOL76/status/1812490624785887379          25   
202    https://x.com/MBComunidad/status/1807880119349...           4   
...                                                  ...         ...   
20332   https://x.com/Fnac_CAT/status/878178336211480576           7   
22391  https://x.com/Candeloide/status/62007071356789...          10   
22761  https://x.com/jamiecullum/status/6198361887535...           6   
24043  https://x.com/Helenity_/status/487866871677714432           5   
24064  https://x.com/MartaSpons/status/48776051240364...           5   

      author_userName  
63            el_pais  
83      planetaamaral  
156       mbcomunidad  
160

In [10]:
import re
import pandas as pd
import emoji
import os
comments = pd.read_csv(r"C:\Users\migue\OneDrive\Escritorio\UAB_INTELIGENCIA_ARTIFICIAL\Tercer_Any\3B\Synthesis Project II\Limpieza\all_comments_definitivo.csv", encoding="utf-8")

In [11]:
columnas_a_eliminar = [
    'Post Likes', 'Post Replies',"Post Views" ,'Post Reposts','Post Quotes', 
    'Post Bookmarks', 'Is Retweet', "Author Name",
    'Author URL', 'Author Description', 'Author Followers', 'Author Following',
    'Author Posts Count', 'Author Join Date', 'Author Is Verified',
    'Author Profile Image', 'Author Profile Banner',
    'Author Media Count', 'Media URL 1', 'Media URL 2', 'Media URL 3',
    'Media URL 4', 'Media Types', 'Has Media', 'Media Count',
    'Is Reply', 'Is Quote Tweet', 'Is Translatable',
    'Is Editable', 'Edits Remaining'
]

comments = comments.drop(columns=columnas_a_eliminar, errors="ignore")
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2303 entries, 0 to 2302
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               2303 non-null   int64 
 1   Post Text        2281 non-null   object
 2   Post Date        2303 non-null   object
 3   Post URL         2303 non-null   object
 4   Post Language    2303 non-null   object
 5   Author Username  2303 non-null   object
 6   Author Location  2303 non-null   object
 7   Reply To User    2303 non-null   object
dtypes: int64(1), object(7)
memory usage: 144.1+ KB


In [ ]:
# ========= 0. NORMALIZAR fullText (primera letra mayúscula, resto minúsculas) =========

def normalizar_fulltext(texto):
    if pd.isna(texto):
        return texto
    return texto.lower()

comments["Post Text"] = comments["Post Text"].apply(normalizar_fulltext)

# ========= 2. QUITAR DUPLICADOS POR ID =========

comments = comments.drop_duplicates(subset=["id"], keep="first")

# ========= 3. LIMPIAR TEXTOS: EMOJIS Y HASHTAGS =========

def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    texto = re.sub(r'http\S+|www.\S+', '', texto)
    texto = re.sub(r'https\S+|www.\S+', '', texto)
    # Quitar menciones @
    #texto = re.sub(r'@\w+', '', texto)
    # Quitar hashtags
    texto = re.sub(r'#\w+', '', texto)
    texto = emoji.replace_emoji(texto, replace='')
    # Quitar espacios extra
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

# Cambiar nombres de columnas
comments.rename(columns={
    'Post Text': 'fullText',
    'Post Date': 'createdAt',
    'Post URL': 'url',
    'Post Language': 'lang',
    'Author Username': 'author_userName',
    'Author Location': 'author_location',
    # Agrega más si es necesario
}, inplace=True)

comments["fullText"] = comments["fullText"].apply(limpiar_texto)
comments["createdAt"] = pd.to_datetime(comments["createdAt"], errors="coerce")
comments["createdAt"] = comments["createdAt"].dt.strftime("%d/%m/%Y %H:%M")

marcas = [
    "cruillabcn", "barcelonaglobal", "btvnoticies", "lavanguardia", "diaridebcn", 
    "cacaolat", "estrelladamm", "rodalies", "catalannews", "movistar_es", 
    "elpaiscatalunya", "radioudeg", "324cat", "naciodigital", "spotify", 
    "on_economia", "on_economia_cat", "diariregio7", "elpeoriodico", "msn_es", "Cati_Tarragona", "beteve", 
    "MBecerraTour", "nuvol_com", "Canal21Ebre", "MPC_management", "elnacionalcat_e",
    "EPSI_UAB", "mundodeportivo", "cati_tarragona", "elnacionalcat", "los40",
    "CatalunyaPlural", "ClubRACC", "rac105", "AlDiaCat", "bnmallorca",
    "subnoise", "canal21ebre", "ebredigitalcat", "aguaitacat", "ElReferente",
    "actualidad_na", "ElNacionalWeb", "EuropaFMCat", "MWCapital", "laganzuaweb",
    "primerafilacat", 
]

# Convertimos a minúsculas por si acaso
comments["author_userName"] = comments["author_userName"].astype(str).str.lower()
marcas = [m.lower() for m in marcas]
comments["Reply To User"] = comments["Reply To User"].astype(str)

# Filtrar filas:
# 1. Donde "Reply to User" NO sea "Not Available"
# 2. Y donde ese valor esté en la lista de marcas (insensibilidad a mayúsculas)
comments = comments[~(
    (comments["author_userName"].str.lower().isin(marcas)) & 
    (comments["Reply To User"] != "Not Available")
)]

# ========= GUARDAR NUEVO CSV LIMPIO =========

comments.to_csv(r"C:\Users\migue\OneDrive\Escritorio\UAB_INTELIGENCIA_ARTIFICIAL\Tercer_Any\3B\Synthesis Project II\Limpieza\all_comments_1.0.csv", encoding="utf-8", index=False)
print("CSV limpio guardado con éxito.")

C:\Users\migue\AppData\Local\Temp\ipykernel_6212\1683321105.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  comments["createdAt"] = pd.to_datetime(comments["createdAt"], errors="coerce")


CSV limpio guardado con éxito.
